In [ ]:
!pip install imageio_ffmpeg
!pip install minio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.6 MB/s 


In [ ]:
import os
import uuid
import time
import pandas as pd
from tqdm import tqdm

import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from minio import Minio
from minio.error import S3Error

import psycopg2
from psycopg2 import OperationalError


In [ ]:
MINIO_ENDPOINT_URL = 
MINIO_ACCESS_KEY =
MINIO_SECRET_KEY = 
MINIO_BUCKET =

client = Minio(
        MINIO_ENDPOINT_URL,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
)

def add_file_to_s3(input_path, object_name):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fput_object(MINIO_BUCKET, object_name, input_path)

def get_file_from_s3(object_name, file_path):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fget_object(MINIO_BUCKET, object_name, file_path)

In [ ]:
POSTGRES_USER=
POSTGRES_PASSWORD=
POSTGRES_DB=
POSTGRES_HOST=
POSTGRES_PORT=

def pg_create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(database=db_name,
                                      user=db_user,
                                      password=db_password,
                                      host=db_host,
                                      port=db_port,)
        connection.autocommit = True
    except OperationalError as ex:
        print(f"The ERROR {ex} occurred")
    return connection

connection = pg_create_connection(POSTGRES_DB, 
                                  POSTGRES_USER, 
                                  POSTGRES_PASSWORD, 
                                  POSTGRES_HOST,
                                  POSTGRES_PORT)

def pg_insert_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_one_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            hander = cursor.fetchone()[0]
            return hander
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_many_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            return cursor.fetchall()
        except OperationalError as ex:
          print(f"The ERROR {ex} occurred")

def add_clip_to_db(video_id, action_type, stime, clip_path):
  action_id = pg_one_select_query(query=f"""
                                  SELECT id FROM public.action_types
                                  WHERE name=%s;
                                  """, param=(action_type,))
  query = f"""
      INSERT INTO events(id_video, id_action, time, clip_path)
      VALUES (%s, %s, %s, %s)
      RETURNING id;
  """
  pg_insert_query(query=query,
                  param=(video_id, action_id, stime, clip_path))

In [ ]:
TEMP_DIR = "/content/tmp_folder"

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)

In [ ]:
clips_paths = pg_many_select_query(query=f"SELECT id, clip_path FROM public.events;")

In [ ]:
clips_paths

[(4, '1606b0e6_0/743b8ab1-9219-4b7c-bdec-075156faa98d.h264'),
 (5, '1606b0e6_0/6557ad2d-8cb2-4b4b-af87-57b5f90fe2c2.h264'),
 (6, '1606b0e6_0/8b391b04-a53f-4e56-afeb-30adc67f5c8f.h264'),
 (7, '1606b0e6_0/b7cd26d8-5eee-4244-a9aa-904c71c5f4db.h264'),
 (8, '1606b0e6_0/200c6a6d-c678-46d5-88c6-5e5a028a21b2.h264'),
 (9, '1606b0e6_0/a7ee4153-4ede-491b-9dcc-b64514d03070.h264'),
 (10, '1606b0e6_0/9ca7e7bf-b1c9-4f4a-bce3-2102c65b3da2.h264'),
 (11, '1606b0e6_0/24ac8ce9-39d8-4984-9f8d-9e207d04debe.h264'),
 (12, '1606b0e6_0/450c3a32-a039-4817-bd97-53b09d5cc872.h264'),
 (13, '1606b0e6_0/370f9471-08ef-4ef3-b4c1-f90d529b23ab.h264'),
 (14, '1606b0e6_0/2cdfa4f4-0d13-48a6-8b15-bd5cd351edfc.h264'),
 (15, '1606b0e6_0/c300a2a3-b52a-4a85-8508-7981aa17a149.h264'),
 (16, '1606b0e6_0/1db86610-483e-4d7b-a86f-2139cb70596d.h264'),
 (17, '1606b0e6_0/9be21d39-7afd-4380-8e9e-fd67f41fd299.h264'),
 (18, '1606b0e6_0/4b124c3f-47b6-4138-9726-d0607e45a880.h264'),
 (19, '1606b0e6_0/1e6118f5-e3d8-47eb-ad9f-8658a8bf94d2.h264')

In [ ]:
for path in tqdm(clips_paths):
  id, object_name = path
  video_name = object_name.split("/")[0]
  input_video_fname = os.path.basename(object_name)
  input_video_name = input_video_fname.split('.')[0]

  tmp_file_path_h264 = f"{TEMP_DIR}/{input_video_fname}"
  tmp_file_path_mp4 = f"{TEMP_DIR}/{input_video_name}.mp4"
  get_file_from_s3(object_name, tmp_file_path_h264)

  convert_command =  f"ffmpeg -i {tmp_file_path_h264} -c:v copy -f mp4 {tmp_file_path_mp4}"
  process = subprocess.run(convert_command.split(), stdout=subprocess.PIPE)

  add_file_to_s3(tmp_file_path_mp4, f'{video_name}/{input_video_name}.mp4')

  query =   """
    UPDATE public.events
    SET clip_path = %s
    WHERE id = %s;
  """
  pg_insert_query(query, param=(f'{video_name}/{input_video_name}.mp4', id)) 

  os.remove(filename_preview)

100%|██████████| 62/62 [01:06<00:00,  1.08s/it]
